In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr

<H1>SPEED DETECTOR BOARDS RECOGNIZED USING CV2 AND ORB MODELS AVAILABLE<H1>

In [2]:
import pandas as pd

<P>DEFINING THE TEMPLATE DIRECTORIES AND MILESTONES CONTAINING SPEED DETECTORS</P>

In [108]:
template_dir = './DataSet/Template images'
mile1_path='./DataSet/Milestone 1'
mile2_path='./DataSet/Milestone 2'

<p>GET SPEED BOARD FUNCTION FOR GETTING THE SPEED WITH THE TEMPLATE using template matching features,Pre-processing the image by tilt angles and contrast brightness</p>

In [128]:
def get_speed_board(template_dir,mile_path,filename):
    # Load the target image and preprocess
    target_path = os.path.join(mile_path, filename)
    img2 = cv2.imread(target_path, 0)  # Target image
    target_width = 800
    img2 = cv2.resize(img2, (target_width, int(img2.shape[0] * target_width / img2.shape[1])))
    
    # Sharpening kernel
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    img2 = cv2.filter2D(img2, -1, kernel)
    image_brightness = np.mean(img2)
    image_contrast = np.std(img2)
    
    # Initialize the ORB detector
    orb = cv2.ORB_create()
    
    best_match = []
    best_img_match = None
    best_template_name = None
    best_avg_distance = float('inf')
    
    # Iterate over all templates
    for template_name in os.listdir(template_dir):
        template_path = os.path.join(template_dir, template_name)
        img1 = cv2.imread(template_path, 0)  # Template image
        
        if img1 is None:
            continue
        
        # Preprocess the template image
        img1 = cv2.filter2D(img1, -1, kernel)
       
    
        # Find the keypoints and descriptors with ORB
        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)
        
        if des1 is None or des2 is None:
            continue
        
        # Create a BFMatcher object and match descriptors
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)
        # Sort matches by distance
        matches = sorted(matches, key=lambda x: x.distance)
        
        # Extract matched keypoints
        points1 = np.zeros((len(matches), 2), dtype=np.float32)
        points2 = np.zeros((len(matches), 2), dtype=np.float32)
        
        for i, match in enumerate(matches):
            points1[i, :] = kp1[match.queryIdx].pt
            points2[i, :] = kp2[match.trainIdx].pt
        
        # Find homography
        h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
        
        # Warp the template image to align with the target image
        height, width = img2.shape
        aligned_template = cv2.warpPerspective(img1, h, (width, height))
        
        # Apply the sharpening filter to the aligned template
        aligned_template = cv2.filter2D(aligned_template, -1, kernel)
        
        # Find the keypoints and descriptors with ORB for the aligned template
        kp1_aligned, des1_aligned = orb.detectAndCompute(aligned_template, None)
        
        if des1_aligned is None or des2 is None:
            continue
        
        # Match descriptors again with the aligned template
        matches_aligned = bf.match(des1_aligned, des2)
        
        # Sort matches by distance
        matches_aligned = sorted(matches_aligned, key=lambda x: x.distance)
        template_brightness = np.mean(img1)
        template_contrast = np.std(img1)
        if image_contrast != 0:
        # Adjust contrast
            img2_brighted = (template_contrast / image_contrast) * (img2 - image_brightness) + template_brightness
        else:
            img2_brighted = img2.copy()
        img2_brighted = np.clip(img2_brighted, 0, 255).astype(np.uint8)
        
        # Find the keypoints and descriptors with ORB for the aligned template
        kp2_aligned, des2_aligned = orb.detectAndCompute(img2_brighted, None)
        
        if des2_aligned is None or des1_aligned is None:
            continue
        # Match descriptors again with the aligned template
        matches_aligned2 = bf.match(des1_aligned, des2_aligned)
        
        # Sort matches by distance
        matches_aligned2 = sorted(matches_aligned2, key=lambda x: x.distance)
        # Calculate average distance of matches
        avg_distance = np.mean([match.distance for match in matches_aligned2])
        
        # Update the best match if current matches are better
        if avg_distance>60:
            continue
        if avg_distance < best_avg_distance:
            best_match = matches_aligned
            best_template_name = template_name
            best_avg_distance = avg_distance
            best_img_match = cv2.drawMatches(aligned_template, kp1_aligned, img2_brighted, kp2_aligned, matches_aligned2, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Show the matches
    result=""
    if best_img_match is not None:
        print(f"Best matching template: {best_template_name} with avg distance: {best_avg_distance}")
        cv2.imshow("Best Matches", best_img_match)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        result+=best_template_name
    else:
        result+="None"
        print("No good match found.")
    return result


<p>To get the picture and the speed dataset</p>

In [ ]:
def get_dataSet(mile_path):
    datas=[]
    for filename in os.listdir(mile_path):
        template=get_speed_board(template_dir,mile_path,filename)
        print(template)
        if(template != "None"):
            speed=template.replace("Template-","").replace(".jpg","").replace(".PNG","")
        else:
            speed=template
        data=[]
        data.append(filename)
        data.append(speed)
        datas.append(data)
    return datas

<P>let's get the dataset for every milepath and convert to <b>Dataframe using pandas</b></P>

In [131]:
mile1_output=get_dataSet(mile1_path)
mile2_output=get_dataSet(mile2_path)
columns=["INPUT IMAGE","SPEED LIMIT"]
df1=pd.DataFrame(mile1_output,columns=columns)
df2=pd.DataFrame(mile2_output,columns=columns)

Best matching template: Template-45.jpg with avg distance: 37.88333333333333
Template-45.jpg
Best matching template: Template-50.jpg with avg distance: 42.78151260504202
Template-50.jpg
Best matching template: Template-85.jpg with avg distance: 43.92888888888889
Template-85.jpg
Best matching template: Template-85.jpg with avg distance: 46.41232227488152
Template-85.jpg
No good match found.
None
Best matching template: Template-40.jpg with avg distance: 53.43715846994535
Template-40.jpg
Best matching template: Template-15.jpg with avg distance: 48.902061855670105
Template-15.jpg
Best matching template: Template-70.jpg with avg distance: 50.18222222222222
Template-70.jpg
Best matching template: Template-25.jpg with avg distance: 44.70472440944882
Template-25.jpg
Best matching template: Template-55.jpg with avg distance: 53.52427184466019
Template-55.jpg
Best matching template: Template-50.jpg with avg distance: 41.53639846743295
Template-50.jpg
Best matching template: Template-85.jpg wit

<H2>CONVERTING TO CSV OUTPUT FILE RESULT</H2>

In [133]:
df1.to_csv("M1_2_OUTPUT.csv",index=False)
df2.to_csv("M2_OUTPUT.csv",index=False)